# 심리상담 챗봇 만들기

트랜스포머의 인코더 부분을 사용하여 한국어 문장 인코딩(벡터화)

In [ ]:
!pip install -q sentence-transformers

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## SentenceBERT를 이용한 문장 인코딩

https://huggingface.co/jhgan/ko-sroberta-multitask

In [ ]:
encoder = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = encoder.encode(sentences)

print(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.37510476 -0.7733843   0.59277105 ...  0.579235    0.32683465
  -0.6508965 ]
 [-0.09361731 -0.18191531 -0.1923082  ... -0.03165787  0.30412537
  -0.26793632]]


## 데이터셋 로드

In [ ]:
df = pd.read_csv('https://github.com/kairess/mental-health-chatbot/raw/master/wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


## 데이터 전처리

- 필요없는 칼럼 제거
- 챗봇 내용 없는 행 제거

In [ ]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [ ]:
print(len(df))

df = df.dropna()

print(len(df))

5231
1034


## 챗봇의 원리

In [ ]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [ ]:
encoder.encode(df.loc[0, '유저'])

array([-4.80606705e-01, -2.94868916e-01,  4.37900335e-01, -6.40137494e-01,
        3.28671262e-02, -3.42647552e-01, -5.47483787e-02,  1.73056293e-02,
       -4.08221245e-01, -5.06034255e-01, -1.68733492e-01, -3.98677111e-01,
       -1.24776855e-01, -9.71540809e-02, -1.65286317e-01,  5.72613114e-03,
        6.13689050e-02, -1.91312239e-01,  2.53917426e-01, -5.85019410e-01,
       -2.84426183e-01, -2.32035294e-01, -3.27080816e-01,  6.72980547e-02,
       -1.65956353e-05, -4.72336292e-01, -3.60021979e-01,  2.91879714e-01,
       -6.63861156e-01, -3.10574740e-01,  5.79524934e-01, -3.11722696e-01,
        1.47696463e-02, -2.12172672e-01,  2.22058132e-01, -1.73828945e-01,
       -3.78458500e-01, -4.20397937e-01, -2.38218680e-01,  6.38704896e-02,
       -1.15304470e-01, -2.44563982e-01, -5.00228286e-01,  1.68355361e-01,
       -6.58360124e-01, -8.91942143e-01, -6.26956880e-01, -3.21965367e-01,
       -7.05358207e-01,  3.71447176e-01, -5.45803547e-01,  7.76298717e-02,
        1.09864406e-01,  

## 모든 데이터셋 인코딩

In [ ]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(encoder.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806067, -0.29486892, 0.43790033, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561576, -0.1450622, 0.29490334, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520035, -0.08126804, 1.0945567, 0.105791..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.767906, 0.4652072, 0.5285069, -0.5076045, ..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277731, -0.3741389, 0.040532075, -0.8624..."


## 예제 문장 입력

In [ ]:
text = '요즘 남편이 비트코인도 하고 속을 너무 썩이네'

embedding = encoder.encode(text)

embedding

array([-5.45266926e-01, -2.66142905e-01,  1.14704776e+00, -4.76195335e-01,
       -1.75621137e-01, -9.16792359e-03,  2.09414408e-01,  7.29714453e-01,
       -2.99766511e-01, -1.03593063e+00,  2.18684539e-01, -1.88601702e-01,
       -3.43514293e-01,  1.46189898e-01,  3.62853147e-02, -1.44428357e-01,
        1.39932662e-01, -3.71749252e-01,  2.08317991e-02, -1.30396593e+00,
       -4.00712103e-01,  1.87196240e-01, -5.06442189e-02,  1.91218048e-01,
       -3.45172793e-01, -5.71000651e-02, -1.56859413e-01, -1.25158727e-01,
       -3.28306258e-02,  4.19633448e-01,  4.53279644e-01, -5.84414363e-01,
       -6.56958699e-01, -8.83868098e-01,  1.48711532e-01,  1.13773301e-01,
       -3.09697092e-01, -3.99329245e-01, -5.68692327e-01, -2.53292203e-01,
       -3.43493998e-01, -3.80637646e-01, -3.72345507e-01, -1.02673396e-01,
       -2.64394641e-01, -1.37276661e+00, -1.77115679e-01,  6.35315850e-02,
       -5.16000926e-01, -3.06972209e-02, -3.99468958e-01, -1.75108343e-01,
        4.17406738e-01,  

## 입력된 문장에 대해 유사도 계산

In [ ]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,similarity
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806067, -0.29486892, 0.43790033, -0.64013...",0.596157
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561576, -0.1450622, 0.29490334, -0.673949...",0.337507
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520035, -0.08126804, 1.0945567, 0.105791...",0.228316
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.767906, 0.4652072, 0.5285069, -0.5076045, ...",0.341304
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277731, -0.3741389, 0.040532075, -0.8624...",0.295595


## 유사도가 가장 높은 답변을 채택

In [ ]:
answer = df.loc[df['similarity'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['similarity'])

구분 배경/남편/갈등
유사한 질문 남편이랑 요즘 사이가 안 좋아요.
챗봇 답변 남편 분과 갈등을 겪고 계시군요. 마음이 많이 상하셨겠어요.
유사도 0.637890100479126


## 심리상담 챗봇 앱

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

def greet(user):
    embedding = encoder.encode(user)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    return answer['챗봇']

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://09bf4457642d206c1c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://09bf4457642d206c1c.gradio.live


In [ ]:
import gradio as gr

def greet(user, history=None):
    if history is None:
        history = []
    embedding = encoder.encode(user)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    history.append([user, answer['챗봇']])

    return history, history

demo = gr.Interface(fn=greet, inputs=["text", "state"], outputs=["chatbot", "state"])

demo.launch(debug=True, share=True)